In [1]:
#Importing important modules
import keras
from keras.datasets import mnist
from keras.models import Sequential
from __future__ import print_function
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from sklearn import datasets
from skimage import exposure
import numpy as np
#import imutils
import cv2
import matplotlib.pyplot as plt
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import os
os.listdir('.')

['.config', 'drive', 'sample_data']

In [0]:
import os
os.chdir('/content/drive/My Drive/My Colab Notebooks/')

Load SVHN data from a h5 file

In [0]:
import h5py
import numpy as np

# Open the file as readonly. The file should be present inside a directory called "data" in the same folder as code
h5f = h5py.File('SVHN_single_grey1.h5', 'r')

# Load the training, test and validation set
x_train = h5f['X_train'][:]
y_train = h5f['y_train'][:]
x_test = h5f['X_test'][:]
y_test = h5f['y_test'][:]


# Close this file
h5f.close()

In [6]:
print("x_train shape", x_train.shape)
print("x_test shape", x_test.shape)
print("y_train shape", y_train.shape)
print("y_test shape", y_test.shape)

x_train shape (42000, 32, 32)
x_test shape (18000, 32, 32)
y_train shape (42000,)
y_test shape (18000,)


In [0]:
print("x_train type", x_train.dtype)
print("y_train type", y_train.dtype)

x_train type float32
y_train type uint8


In [0]:
#since the original data is already split into train and test data, we will split the train data into train and validation data
# we will take validation data as 10% of the above training data
(x_train, x_val, y_train, y_val) = train_test_split(x_train, y_train,test_size=0.1, random_state=10)

In [0]:
print("training data points: {}".format(len(y_train)))
print("validation data points: {}".format(len(y_val)))
print("testing data points: {}".format(len(y_test)))

training data points: 37800
validation data points: 4200
testing data points: 18000


In [0]:
# convert train data and validation data into a 2D array so that it can be fed to KNN model
nsamples, nx, ny = x_train.shape
d2_x_train = x_train.reshape((nsamples,nx*ny))
nsamples, nx, ny = x_val.shape
d2_x_val = x_val.reshape((nsamples,nx*ny))

In [0]:
print(d2_x_train.shape)
print(d2_x_val.shape)

(37800, 1024)
(4200, 1024)


## Use KNN classifier 

In [0]:
# initialize the values of k for our k-Nearest Neighbor classifier along with the
# list of accuracies for each value of k
# I have run a range of k values from 1 to 100 and this range seems to give the best accuracy for this data set. 
accuracies = []

# loop over various values of `k` for the k-Nearest Neighbor classifier

for k in range(23, 31, 2):
          # train the k-Nearest Neighbor classifier with the current value of `k`
          model = KNeighborsClassifier(n_neighbors=k)
          model.fit(d2_x_train, y_train)
          # evaluate the model and update the accuracies list
          score = model.score(d2_x_val, y_val)
          print("k=%d, accuracy=%.2f%%" % (k, score * 100))
          accuracies.append(score)
          
# find the value of k that has the largest accuracy

k=23, accuracy=51.33%
k=25, accuracy=51.52%
k=27, accuracy=51.38%
k=29, accuracy=51.17%


In [0]:
kVals = range(23, 31, 2)
i = np.argmax(accuracies)
print("k=%d achieved highest accuracy of %.2f%% on validation data" % (kVals[i],accuracies[i] * 100))

k=25 achieved highest accuracy of 51.52% on validation data


In [0]:
# convert test data also into a 2D array so that it can be fed to KNN model
nsamples, nx, ny = x_test.shape
d2_x_test = x_test.reshape((nsamples,nx*ny))
d2_x_test.shape

(18000, 1024)

In [0]:
# re-train our classifier using the best k value and predict the labels of the
# test data

model = KNeighborsClassifier(n_neighbors=kVals[i])
model.fit(d2_x_train, y_train)
predictions = model.predict(d2_x_test)

In [0]:
print("EVALUATION ON TESTING DATA")
print(classification_report(y_test, predictions))

EVALUATION ON TESTING DATA
              precision    recall  f1-score   support

           0       0.44      0.69      0.54      1814
           1       0.44      0.73      0.55      1828
           2       0.62      0.53      0.57      1803
           3       0.46      0.41      0.43      1719
           4       0.64      0.64      0.64      1812
           5       0.51      0.40      0.45      1768
           6       0.51      0.40      0.45      1832
           7       0.71      0.63      0.67      1808
           8       0.47      0.37      0.41      1812
           9       0.54      0.43      0.48      1804

    accuracy                           0.52     18000
   macro avg       0.53      0.52      0.52     18000
weighted avg       0.53      0.52      0.52     18000



In [0]:
print ("Confusion matrix")
print(confusion_matrix(y_test,predictions))

Confusion matrix
[[1246   63   32   43   56   37  106   46   73  112]
 [ 105 1338   60   76   73   33   31   46   29   37]
 [  96  260  948   98   51   44   31  144   49   82]
 [ 118  290  117  705   55  176   39   57   97   65]
 [ 102  271   40   45 1154   23   62   21   37   57]
 [ 157  169   56  219   77  702  133   36  129   90]
 [ 318  128   33   50  151  135  737   18  213   49]
 [ 101  200  108   79   42   34   37 1137   26   44]
 [ 271  131   61  108   91  103  231   27  665  124]
 [ 328  158   68  100   63   82   52   68  111  774]]


## Now use Deep neural network to classify the images

In [0]:
import h5py
import numpy as np

# Open the file as readonly. The file should be present inside a directory called "data" in the same folder as code
h5f = h5py.File('SVHN_single_grey1.h5', 'r')

# Load the training, test and validation set
x_train = h5f['X_train'][:]
y_train = h5f['y_train'][:]
x_test = h5f['X_test'][:]
y_test = h5f['y_test'][:]


# Close this file
h5f.close()

In [8]:
print("x_train shape", x_train.shape)
print("x_test shape", x_test.shape)
print("y_train shape", y_train.shape)
print("y_test shape", y_test.shape)

x_train shape (42000, 32, 32)
x_test shape (18000, 32, 32)
y_train shape (42000,)
y_test shape (18000,)


In [9]:
x_train = x_train.reshape(x_train.shape[0], 32, 32, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 32, 32, 1).astype('float32')
x_train /= 255
x_test /= 255
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

print('--- THE DATA ---')
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

--- THE DATA ---
x_train shape: (42000, 32, 32, 1)
42000 train samples
18000 test samples


In [0]:
BATCH_SIZE = 256
EPOCHS = 50

In [53]:
    # Define the Type of Model
    model = Sequential()

    # Flatten Imgaes to Vector
    model.add(Reshape((1024,), input_shape=(32, 32, 1)))

    # Layer 1
    model.add(Dense(output_dim=2048, init='he_normal', bias=True))
    #model.add(BatchNormalization())
    model.add(Activation("relu"))

    #model.add(Dropout(0.2))
    
    #Layer 2
    model.add(Dense(output_dim=1024, init='he_normal', bias=True))
    #model.add(BatchNormalization())
    model.add(Activation("relu"))

    #model.add(Dropout(0.2))

    # Layer 3
    model.add(Dense(output_dim=10, init='he_normal', bias=True))
    model.add(Activation("softmax"))

    opt = keras.optimizers.Adam(lr=0.0005)

    # Loss and Optimizer
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    ## Store Training Results
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, mode='auto')
    callback_list = [early_stopping]# [stats, early_stopping]

    # Train the model
    model.fit(x_train, y_train, nb_epoch=EPOCHS, batch_size=BATCH_SIZE,
              validation_data=(x_test, y_test), verbose=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=2048, kernel_initializer="he_normal", use_bias=True)`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1024, kernel_initializer="he_normal", use_bias=True)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=10, kernel_initializer="he_normal", use_bias=True)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 42000 samples, validate on 18000 samples
Epoch 1/50
42000/42000 [==============================] - 32s 766us/step - loss: 2.0577 - acc: 0.2914 - val_loss: 1.6615 - val_acc: 0.4341
Epoch 2/50
42000/42000 [==============================] - 30s 715us/step - loss: 1.3334 - acc: 0.5800 - val_loss: 1.3207 - val_acc: 0.5901
Epoch 3/50
42000/42000 [==============================] - 30s 712us/step - loss: 1.0953 - acc: 0.6634 - val_loss: 1.0766 - val_acc: 0.6632
Epoch 4/50
42000/42000 [==============================] - 30s 712us/step - loss: 0.9793 - acc: 0.7002 - val_loss: 0.9943 - val_acc: 0.6953
Epoch 5/50
42000/42000 [==============================] - 30s 710us/step - loss: 0.8875 - acc: 0.7269 - val_loss: 0.8997 - val_acc: 0.7202
Epoch 6/50
42000/42000 [==============================] - 30s 708us/step - loss: 0.8295 - acc: 0.7475 - val_loss: 0.8869 - val_acc: 0.7290
Epoch 7/50
42000/42000 [==============================] - 30s 709us/step - loss: 0.7714 - acc: 0.7662 - val_loss: 0.

In [75]:
  loss, accuracy = model.evaluate(x_test, y_test)
  print("test loss = ",loss)
  print("test accuracy", accuracy)

18000/18000 [==============================] - 5s 272us/step
test loss =  0.5712687921457821
test accuracy 0.8530555555555556


## Print the classification report

In [0]:
predictions = model.predict(x_test)

predictions = (predictions > 0.5) 

In [87]:
print("EVALUATION ON TESTING DATA")
print(classification_report(y_test, predictions))

EVALUATION ON TESTING DATA
              precision    recall  f1-score   support

           0       0.96      0.57      0.72      1814
           1       0.94      0.48      0.64      1828
           2       0.91      0.72      0.81      1803
           3       0.78      0.72      0.75      1719
           4       0.48      0.95      0.64      1812
           5       0.94      0.55      0.69      1768
           6       0.82      0.73      0.77      1832
           7       0.98      0.59      0.73      1808
           8       0.95      0.50      0.65      1812
           9       0.59      0.87      0.70      1804

   micro avg       0.75      0.67      0.71     18000
   macro avg       0.83      0.67      0.71     18000
weighted avg       0.83      0.67      0.71     18000
 samples avg       0.67      0.67      0.67     18000



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Grid Search

In [0]:
import numpy as np
from keras.constraints import maxnorm
from keras import backend as keras_backend
from keras.utils import np_utils

from keras.models import load_model

from sklearn.model_selection import GridSearchCV, cross_val_score

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier


from sklearn.model_selection import StratifiedKFold

random_seed = 64
np.random.seed(random_seed)


Using TensorFlow backend.


In [0]:
import h5py
import numpy as np

# Open the file as readonly. The file should be present inside a directory called "data" in the same folder as code
h5f = h5py.File('SVHN_single_grey1.h5', 'r')

# Load the training, test and validation set
x_train = h5f['X_train'][:]
y_train = h5f['y_train'][:]
x_test = h5f['X_test'][:]
y_test = h5f['y_test'][:]


# Close this file
h5f.close()

In [0]:

x_train = x_train.reshape(x_train.shape[0], 32, 32, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 32, 32, 1).astype('float32')
x_train /= 255
x_test /= 255
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

original_y_train = y_train
original_y_test = y_test

print ('Train size:', x_train.shape[0])
print ('Test size:', x_test.shape[0])


Train size: 42000
Test size: 18000


In [0]:
# Function to create model, required for KerasClassifier
def create_model(number_of_layers=2, neurons_per_layer = 32, dropout_percent = 0.2, optimizer='adam'):
    # create model
    model = Sequential()
    
    # Flatten Imgaes to Vector
    model.add(Reshape((1024,), input_shape=(32, 32, 1)))

    model.add(Dense(output_dim=neurons_per_layer, activation='relu', init='he_normal', bias=True))

    for i in range(number_of_layers-1):
        model.add(Dense(output_dim=neurons_per_layer, activation='relu', kernel_constraint = maxnorm(3)))
        model.add(Dropout(dropout_percent))

    # Layer 2
    model.add(Dense(output_dim=10, init='he_normal', bias=True))
    model.add(Activation("softmax"))
            
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
   
    return model

In [0]:
# create model
kc_model = KerasClassifier(build_fn=create_model,number_of_layers=2, neurons_per_layer = 1024, optimizer = 'adam', epochs=20, batch_size=256, verbose=0) # Wrapper for scikitlearn API, provides 
                                                                                    # facility to get scores 


In [0]:
# define the grid search parameters

param_grid = dict(number_of_layers = [2 , 3],
                  batch_size = [128, 256, 512],
                 neurons_per_layer = [512 , 1024],
                 optimizer = ['adam'])


In [0]:
gridsearcher = GridSearchCV(estimator=kc_model, param_grid=param_grid, n_jobs=1)
grid_result = gridsearcher.fit(x_train, original_y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=512, kernel_initializer="he_normal", use_bias=True)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_constraint=<keras.con..., units=512)`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=10, kernel_initializer="he_normal", use_bias=True)`
  from ipykernel import kernelapp as app


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=1024, kernel_initializer="he_normal", use_bias=True)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_constraint=<keras.con..., units=1024)`
  # This is added back by InteractiveShellApp.init_path()


Best: 0.802286 using {'batch_size': 512, 'neurons_per_layer': 1024, 'number_of_layers': 3, 'optimizer': 'adam'}
0.783071 (0.007574) with: {'batch_size': 128, 'neurons_per_layer': 512, 'number_of_layers': 2, 'optimizer': 'adam'}
0.782690 (0.011226) with: {'batch_size': 128, 'neurons_per_layer': 512, 'number_of_layers': 3, 'optimizer': 'adam'}
0.790167 (0.007110) with: {'batch_size': 128, 'neurons_per_layer': 1024, 'number_of_layers': 2, 'optimizer': 'adam'}
0.797143 (0.012639) with: {'batch_size': 128, 'neurons_per_layer': 1024, 'number_of_layers': 3, 'optimizer': 'adam'}
0.780667 (0.012278) with: {'batch_size': 256, 'neurons_per_layer': 512, 'number_of_layers': 2, 'optimizer': 'adam'}
0.790214 (0.008417) with: {'batch_size': 256, 'neurons_per_layer': 512, 'number_of_layers': 3, 'optimizer': 'adam'}
0.784786 (0.009060) with: {'batch_size': 256, 'neurons_per_layer': 1024, 'number_of_layers': 2, 'optimizer': 'adam'}
0.800619 (0.008775) with: {'batch_size': 256, 'neurons_per_layer': 1024, 

## Adding batch normalization for the FC layer

In [0]:
from keras.layers import BatchNormalization

In [0]:
import h5py
import numpy as np

# Open the file as readonly. The file should be present inside a directory called "data" in the same folder as code
h5f = h5py.File('SVHN_single_grey1.h5', 'r')

# Load the training, test and validation set
x_train = h5f['X_train'][:]
y_train = h5f['y_train'][:]
x_test = h5f['X_test'][:]
y_test = h5f['y_test'][:]


# Close this file
h5f.close()

In [77]:
print("x_train shape", x_train.shape)
print("x_test shape", x_test.shape)
print("y_train shape", y_train.shape)
print("y_test shape", y_test.shape)



x_train shape (42000, 32, 32)
x_test shape (18000, 32, 32)
y_train shape (42000,)
y_test shape (18000,)


In [78]:
x_train = x_train.reshape(x_train.shape[0], 32, 32, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 32, 32, 1).astype('float32')
x_train /= 255
x_test /= 255
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

print('--- THE DATA ---')
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


--- THE DATA ---
x_train shape: (42000, 32, 32, 1)
42000 train samples
18000 test samples


In [0]:
from keras import optimizers

BATCH_SIZE = 256
EPOCHS = 20

In [81]:
    # Define the Type of Model
    model = Sequential()

    # Flatten Imgaes to Vector
    model.add(Reshape((1024,), input_shape=(32, 32, 1)))

    # Layer 1
    model.add(Dense(output_dim=2048, init='he_normal', bias=False))
    model.add(BatchNormalization())
    model.add(Activation("relu"))

    #model.add(Dropout(0.2))
    
    #Layer 2
    model.add(Dense(output_dim=1024, init='he_normal', bias=False))
    model.add(BatchNormalization())
    model.add(Activation("relu"))

    #model.add(Dropout(0.2))

    # Layer 3
    model.add(Dense(output_dim=10, init='he_normal', bias=True))
    model.add(Activation("softmax"))

    opt = keras.optimizers.Adam(lr=0.0005)

    # Loss and Optimizer
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    ## Store Training Results
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, mode='auto')
    callback_list = [early_stopping]# [stats, early_stopping]

    # Train the model
    model.fit(x_train, y_train, nb_epoch=EPOCHS, batch_size=BATCH_SIZE,
              validation_data=(x_test, y_test), verbose=True)
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=2048, kernel_initializer="he_normal", use_bias=False)`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1024, kernel_initializer="he_normal", use_bias=False)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=10, kernel_initializer="he_normal", use_bias=True)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 42000 samples, validate on 18000 samples
Epoch 1/20
42000/42000 [==============================] - 33s 789us/step - loss: 1.1914 - acc: 0.6258 - val_loss: 1.6403 - val_acc: 0.4805
Epoch 2/20
42000/42000 [==============================] - 31s 738us/step - loss: 0.7403 - acc: 0.7746 - val_loss: 2.1924 - val_acc: 0.4474
Epoch 3/20
42000/42000 [==============================] - 31s 736us/step - loss: 0.6083 - acc: 0.8131 - val_loss: 1.7353 - val_acc: 0.5413
Epoch 4/20
42000/42000 [==============================] - 31s 741us/step - loss: 0.5593 - acc: 0.8284 - val_loss: 3.0890 - val_acc: 0.3918
Epoch 5/20
42000/42000 [==============================] - 34s 812us/step - loss: 0.4922 - acc: 0.8501 - val_loss: 1.4757 - val_acc: 0.5909
Epoch 6/20
42000/42000 [==============================] - 32s 754us/step - loss: 0.4455 - acc: 0.8646 - val_loss: 2.1714 - val_acc: 0.4936
Epoch 7/20
42000/42000 [==============================] - 32s 756us/step - loss: 0.4141 - acc: 0.8731 - val_loss: 1.

## Observations:

a. Tried out different hyperparameters like 
  1. batch_size ranging from 32 to 30000. Seems like 256 is giving better results for this data set.
  2. learning_rate ranging from 0.05 to 0.0005. Seems like 0.001 is giving better results.
  3. epochs ranging from 10 to 1000. Seems like anything above 50 the accuracy does not vary much.
  4. number of layers ranging from 1 to 4. Seems like 2 layers is giving better accuracy.
  5. number of neurons in each layer - tried different combinations in each layer.2048 in 1st layer and 1024 in 2nd layer seems to give better accuracy. 

b. Batch normalization does not seem to add any benefit in this scenario. Actually it is making the test accuracy go down from 85% to almost 70%.

c. Tried out different combinations of dropout layers, but that also did not add any impact to the accuracy score. 

d. Higher batch_size was giving very less accuracy in this case. 

e. Grid search cv also did not give higher accuracy than the normal case. 

